# Getting the Last Goat Horn

I've gone a little goat crazy breeding goats on a mountain, but hey--I've got all but the last goat horn, so I'd say things are going pretty well.

But now that it's been confirmed that each of any given goat's horns will produce the same sound, then that means the information encoding that must be embedded somewhere in the goat's data.

## Imports, Setup and Macros

In [1]:
import json
from collections import defaultdict
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, Tuple

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size: float = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (17.0 KB)
- level.dat_old (17.0 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)


In [5]:
def get_chunk_and_region_from_position(
    x: float, y: float | None = None, z: float = 0
) -> Tuple[Path, Tuple[int, int]]:
    """Given a position, return the corresponding region file
    and chunk

    Parameters
    ----------
    x : int or float
        The x coordinate
    y :
        This will be ignored but is provided to allow
        for splatting a three-coordinate
    z : int or float
        The z coordinate

    Returns
    -------
    Path
        The path to the region file containing this coordinate
    (int, int) tuple
        The chunk index containing this coordinate
    """
    chunk = (int(x // 16), int(z // 16))
    region = (int(chunk[0] // 32), int(chunk[1] // 32))
    return save_folder / "region" / "r.{0}.{1}.mca".format(*region), chunk

In [6]:
level = nbt.NBTFile(save_folder / "level.dat")
position = tuple(int(coord.value) for coord in level["Data"]["Player"]["Pos"])
neighborhood = [
    get_chunk_and_region_from_position(*pos)
    for pos in (
        (position[0] - 200, position[1], position[2] - 200),
        (position[0] - 200, position[1], position[2] + 200),
        (position[0] + 200, position[1], position[2] - 200),
        (position[0] + 200, position[1], position[2] + 200),
    )
]
region_files = set(region for region, _ in neighborhood)

display(Markdown(f"I'm at coordinates {position}"))
display(
    Markdown(
        "and so should look through chunks"
        f" ({neighborhood[0][1][0]} – {neighborhood[3][1][0]},"
        f" {neighborhood[0][1][1]} – {neighborhood[3][1][1]})"
        "\nin region files:"
    )
)
for region_file in region_files:
    display(Markdown(f"\n  - {region_file.name}"))

I'm at coordinates (-6293, 213, -1383)

and so should look through chunks (-406 – -381, -99 – -74)
in region files:


  - r.-13.-4.mca


  - r.-12.-3.mca


  - r.-12.-4.mca


  - r.-13.-3.mca

## Get a Goat

In [7]:
for region_file in region_files:
    entity_data = region.RegionFile(save_folder / "entities" / region_file.name)
    for chunk in entity_data.iter_chunks():
        if (
            chunk["Position"][0] < neighborhood[0][1][0]
            or chunk["Position"][0] > neighborhood[3][1][0]
            or chunk["Position"][1] < neighborhood[0][1][1]
            or chunk["Position"][1] > neighborhood[3][1][1]
        ):
            continue
        for entity in chunk["Entities"]:
            if entity["id"].value == "minecraft:goat":
                goat = entity
                summarize_keystore(goat)
                break
        else:
            continue
        break


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `0`
 - `Attributes` : 
	
	 - `0` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
	 - `1` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `ForcedAge` : `4300`
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `FallDistance` : `0.0`
 - `InLove` : `0`
 - `DeathTime` : `0`
 - `IsScreamingGoat` : `0`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `PersistenceRequired` : `0`
 - `id` : `"minecraft:goat"`
 - `UUID` : (4 items)
 - `Age` : `0`
 - `Motion` : (3 items)
 - `HasLeftHorn` : `0`
 - `HasRightHorn` : `0`
 - `Health` : `10.0`
 - `LeftHanded` : `0`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Rotation` : 
	
	 - `0` : `255.92892456054688`
	 - `1` : `0.0`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `ArmorDropChances` : (4 items)
 - `Pos` : (3 items)
 - `Fire` : `-1`
 - `ArmorItems` : (4 items)
 - `CanPickUpLoot` : `0`
 - `HurtTime` : `0`
 - `LoveCause` : (4 items)

Hmm... Attributes?

In [8]:
for attribute in goat["Attributes"]:
    summarize_keystore(attribute)


 - `Base` : `0.20000000298023224`
 - `Name` : `"minecraft:generic.movement_speed"`


 - `Base` : `2.0`
 - `Name` : `"minecraft:generic.attack_damage"`

Nope. But why does a goat have "armor items"?

In [9]:
for attribute in goat["ArmorItems"]:
    summarize_keystore(attribute)

I guess those are just placeholder spots?

Grasping at straws now:

In [10]:
summarize_keystore(goat["Brain"]["memories"])


 - `minecraft:is_tempted` : 
	
	 - `0` : `"value"`
 - `minecraft:ram_cooldown_ticks` : 
	
	 - `0` : `"value"`
 - `minecraft:long_jump_cooling_down` : 
	
	 - `0` : `"value"`

Let me check some more goats.

In [11]:
for region_file in region_files:
    entity_data = region.RegionFile(save_folder / "entities" / region_file.name)
    for chunk in entity_data.iter_chunks():
        if (
            chunk["Position"][0] < neighborhood[0][1][0]
            or chunk["Position"][0] > neighborhood[3][1][0]
            or chunk["Position"][1] < neighborhood[0][1][1]
            or chunk["Position"][1] > neighborhood[3][1][1]
        ):
            continue
        for entity in chunk["Entities"]:
            if entity["id"].value == "minecraft:goat":
                goat = entity
                if goat["HasLeftHorn"].value != 1:
                    summarize_keystore(goat)
                    break
        else:
            continue
        break


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `0`
 - `Attributes` : 
	
	 - `0` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
	 - `1` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `ForcedAge` : `4300`
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `FallDistance` : `0.0`
 - `InLove` : `0`
 - `DeathTime` : `0`
 - `IsScreamingGoat` : `0`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `PersistenceRequired` : `0`
 - `id` : `"minecraft:goat"`
 - `UUID` : (4 items)
 - `Age` : `0`
 - `Motion` : (3 items)
 - `HasLeftHorn` : `0`
 - `HasRightHorn` : `0`
 - `Health` : `10.0`
 - `LeftHanded` : `0`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Rotation` : 
	
	 - `0` : `255.92892456054688`
	 - `1` : `0.0`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `ArmorDropChances` : (4 items)
 - `Pos` : (3 items)
 - `Fire` : `-1`
 - `ArmorItems` : (4 items)
 - `CanPickUpLoot` : `0`
 - `HurtTime` : `0`
 - `LoveCause` : (4 items)

Okay, so they can have different health values, but if that's how horn sound is determined, that's going to be a pain to decode.

In [12]:
for region_file in region_files:
    entity_data = region.RegionFile(save_folder / "entities" / region_file.name)
    for chunk in entity_data.iter_chunks():
        if (
            chunk["Position"][0] < neighborhood[0][1][0]
            or chunk["Position"][0] > neighborhood[3][1][0]
            or chunk["Position"][1] < neighborhood[0][1][1]
            or chunk["Position"][1] > neighborhood[3][1][1]
        ):
            continue
        for entity in chunk["Entities"]:
            if entity["id"].value == "minecraft:goat":
                goat = entity
                if goat["IsScreamingGoat"].value == 1:
                    summarize_keystore(goat)
                    break
        else:
            continue
        break


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `0`
 - `Attributes` : 
	
	 - `0` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
	 - `1` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `ForcedAge` : `0`
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `FallDistance` : `0.0`
 - `InLove` : `0`
 - `DeathTime` : `0`
 - `IsScreamingGoat` : `1`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `PersistenceRequired` : `0`
 - `id` : `"minecraft:goat"`
 - `UUID` : (4 items)
 - `Age` : `0`
 - `Motion` : (3 items)
 - `HasLeftHorn` : `0`
 - `HasRightHorn` : `1`
 - `Health` : `9.0`
 - `LeftHanded` : `0`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Rotation` : 
	
	 - `0` : `43.822998046875`
	 - `1` : `0.0`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `ArmorDropChances` : (4 items)
 - `Pos` : (3 items)
 - `Fire` : `-1`
 - `ArmorItems` : (4 items)
 - `CanPickUpLoot` : `0`
 - `HurtTime` : `0`

My hypothesis is that I think it's by determined by UUID. But even if it's by something else, it's clearly not going to be easily decipherable.

## Well how many screaming goats are there?

In [13]:
goats: Dict[str, int] = defaultdict(int)


for region_file in region_files:
    entity_data = region.RegionFile(save_folder / "entities" / region_file.name)
    for chunk in entity_data.iter_chunks():
        if (
            chunk["Position"][0] < neighborhood[0][1][0]
            or chunk["Position"][0] > neighborhood[3][1][0]
            or chunk["Position"][1] < neighborhood[0][1][1]
            or chunk["Position"][1] > neighborhood[3][1][1]
        ):
            continue
        for entity in chunk["Entities"]:
            if entity["id"].value == "minecraft:goat":
                goat = entity
                goats["n"] += 1
                if goat["Age"].value != 0:
                    goats["baby"] += 1
                if goat["IsScreamingGoat"].value == 1:
                    goats["screaming"] += 1
                    if (
                        goat["Age"].value == 0
                        and goat["HasLeftHorn"].value == 1
                        and goat["HasRightHorn"].value == 1
                    ):
                        goats["possible_dreams"] += 1
display(Markdown(f"{goats['screaming']} / {goats['n']} are screaming"))
display(
    Markdown(
        f"There are {goats['possible_dreams']} adults right now who could have the final horn."
    )
)

8 / 99 are screaming

There are 1 adults right now who could have the final horn.

Blergh.